# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("city_data.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yellowknife,62.4560,-114.3525,66.72,68,75,11.50,CA,1658972293
1,1,Bluff,-46.6000,168.3333,47.80,87,100,12.66,NZ,1658972142
2,2,Baruun-Urt,46.6806,113.2792,68.25,42,12,2.93,MN,1658972301
3,3,Raudeberg,61.9875,5.1352,45.27,72,20,6.44,NO,1658972302
4,4,Manduria,40.4018,17.6335,78.98,81,0,9.82,IT,1658972302
...,...,...,...,...,...,...,...,...,...,...
562,562,Bahir Dar,11.5936,37.3908,59.45,93,100,3.36,ET,1658973122
563,563,Vikersund,59.9659,9.9950,59.49,63,100,2.24,NO,1658973182
564,564,Domoni,-12.2569,44.5319,73.42,72,1,9.48,KM,1658973183
565,565,Oranjemund,-28.5500,16.4333,51.93,84,0,2.33,NaN,1658973183


In [3]:
# Drop the Unnamed:0 column that was created when the .csv file was read into the notebook
city_data_df.drop(["Unnamed: 0"], axis=1, inplace=True)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,66.72,68,75,11.50,CA,1658972293
1,Bluff,-46.6000,168.3333,47.80,87,100,12.66,NZ,1658972142
2,Baruun-Urt,46.6806,113.2792,68.25,42,12,2.93,MN,1658972301
3,Raudeberg,61.9875,5.1352,45.27,72,20,6.44,NO,1658972302
4,Manduria,40.4018,17.6335,78.98,81,0,9.82,IT,1658972302
...,...,...,...,...,...,...,...,...,...
562,Bahir Dar,11.5936,37.3908,59.45,93,100,3.36,ET,1658973122
563,Vikersund,59.9659,9.9950,59.49,63,100,2.24,NO,1658973182
564,Domoni,-12.2569,44.5319,73.42,72,1,9.48,KM,1658973183
565,Oranjemund,-28.5500,16.4333,51.93,84,0,2.33,NaN,1658973183


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [5]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Fill Nan values and convert to float
humidity = city_data_df["Humidity"].astype(float)

In [6]:
# Customize the size of the figure
figure_layout = {
    "width": "400px",
    "height": "300px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

In [11]:
# Plot heatmap
fig = gmaps.figure(layout = figure_layout)

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                    dissipating=True, max_intensity = 10,
                                    point_radius = 5)

# Add heat layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 75) & (city_data_df["Max Temp"] <= 85)]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Manduria,40.4018,17.6335,78.98,81,0,9.82,IT,1658972302
16,Gorontalo,0.5412,123.0595,84.29,74,42,7.27,ID,1658972305
17,Vaini,-21.2000,-175.2000,80.76,83,20,9.22,TO,1658972140
22,Port Hardy,50.6996,-127.4199,75.38,60,20,9.22,CA,1658972066
27,Sorong,-0.8833,131.2500,81.41,80,15,3.74,ID,1658972308
...,...,...,...,...,...,...,...,...,...
545,Luba,3.4568,8.5547,76.89,89,100,12.37,GQ,1658973119
546,Yatou,37.1500,122.3833,77.36,90,100,9.62,CN,1658973119
547,Mitsamiouli,-11.3847,43.2844,75.54,70,37,5.73,KM,1658973119
554,Akyab,20.1500,92.9000,83.61,76,83,4.65,MM,1658973120


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
# Create dataframe and add empty column
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\ninap\AppData\Local\Temp/ipykernel_33956/215770354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
4,Manduria,IT,40.4018,17.6335,
16,Gorontalo,ID,0.5412,123.0595,
17,Vaini,TO,-21.2000,-175.2000,
22,Port Hardy,CA,50.6996,-127.4199,
27,Sorong,ID,-0.8833,131.2500,
...,...,...,...,...,...
545,Luba,GQ,3.4568,8.5547,
546,Yatou,CN,37.1500,122.3833,
547,Mitsamiouli,KM,-11.3847,43.2844,
554,Akyab,MM,20.1500,92.9000,


In [76]:
# Find the first hotel for each city located within 5,000 meters of the coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"location": "40.4018, 17.6335",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

In [77]:
# Loop through dataframe and add nearby hotel result

# Create a list of coordinates by using iterrows to iterate through hotel_df
for index, row in hotel_df.iterrows():
    
    # Get Lat/Lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
    try:
        print(f"Closest {hotel} hotel is {results[0]["Hotel Name"]}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["Hotel Name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field

In [78]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [81]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,Manduria,IT,40.4018,17.6335,
16,Gorontalo,ID,0.5412,123.0595,
17,Vaini,TO,-21.2000,-175.2000,
22,Port Hardy,CA,50.6996,-127.4199,
27,Sorong,ID,-0.8833,131.2500,
...,...,...,...,...,...
545,Luba,GQ,3.4568,8.5547,
546,Yatou,CN,37.1500,122.3833,
547,Mitsamiouli,KM,-11.3847,43.2844,
554,Akyab,MM,20.1500,92.9000,


In [82]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_locations)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…